In [1]:
import numpy as np
import pandas as pd
import os

os.chdir('/home/jovyan/work/')

In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/gdrive')
  %cd ../gdrive/MyDrive/adsi_a2/
else:
  # set working directory
  os.chdir('/home/jovyan/work/')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/.shortcut-targets-by-id/1ejGfYDC-KnkyvYV7Tc23fm0JxjKhgVaI/adsi_a2


In [3]:
# nominate columns
col_tar = ['beer_style']
col_cat = ['brewery_name']
col_num = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste']

In [4]:
filepath = 'data/processed/beer_review_cleaned.csv'

# check if cleaned file already exist
if os.path.exists(filepath):
    df_cleaned = pd.read_csv(filepath)
else:
    # load data from csv
    df = pd.read_csv('data/raw/beer_reviews.csv')
        
    # clean up dataset: drop unrelated columns and drop rows that contain NA 
    df_cleaned = df.copy()
    df_cleaned = df[col_cat + col_num + col_tar]
    
    # store data
    df_cleaned.dropna(inplace=True)
    df_cleaned.to_csv('data/processed/beer_review_cleaned.csv', index=False)

In [5]:
df_cleaned.shape

(1586599, 6)

In [6]:
from sklearn.model_selection import train_test_split

# extract target label column
target = df_cleaned.pop(col_tar[0])

# split data
X_train, X_test, y_train, y_test = train_test_split(df_cleaned, target, test_size=0.2, random_state=42)

In [7]:
print(f'Total Prediction Categories: {len(set(target))}')

Total Prediction Categories: 104


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

num_tfm = Pipeline(steps=[('scaler', StandardScaler())])
cat_tfm = Pipeline(steps=[('label_encoder', LabelEncoder(sparse=False))])

In [9]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('col_num', num_tfm, col_num),
        ('col_cat', cat_tfm, col_cat)
    ]
)

In [10]:
tst = y_train.reset_index()
tst.shape

(1269279, 2)

In [12]:
X_trn1, X_trn2, y_trn1, y_trn2 = train_test_split(X_train, y_train, stratify=y_train, test_size=0.98, random_state=42)

In [13]:
X_trn = preprocessor.fit_transform(X_trn1)

In [14]:
from sklearn.preprocessing import OrdinalEncoder

target_enc = OrdinalEncoder()

y_trne = target_enc.fit_transform(tst)
y_vale = target_enc.transform(y_train)

ValueError: Expected 2D array, got 1D array instead:
array=['American Double / Imperial Pilsner' 'English India Pale Ale (IPA)'
 'Weizenbock' ... 'Russian Imperial Stout' 'American Amber / Red Ale'
 'English Brown Ale'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [39]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_trn, y=y_trn1)
# val_dataset = PytorchDataset(X=X_val, y=y_val)
# test_dataset = PytorchDataset(X=X_test, y=y_test)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
